# Data Cleaning - Part 01

## Cleaning missing values

Identifying and cleaning NaN values. Later on we can identify outliers using the *IQR + (1.25 * IQR)* formula. 

In [149]:
# Importing data

DATA_PATH = "./df_rides.csv"

import pandas as pd

# Creating DataFrame
df = pd.read_csv(DATA_PATH)
df.head()

,user_gender,user_birthdate,user_residence,ride_date,time_start,time_end,station_start,station_end,ride_duration,ride_late
0,M,1971-06-08,NaN,2018-01-01,06:05:18,06:21:33,11 - Rodoviária 2,41 - Instituto de Artes,16.250000,0.0
1,M,1989-02-11,DF,2018-01-01,06:27:01,06:32:17,26 - Ministério da Saude,28 - CNMP - Conselho Nacional do Ministério Pú...,5.266667,0.0
2,M,1968-07-19,NaN,2018-01-01,06:29:33,06:44:57,11 - Rodoviária 2,43 - Biblioteca Central,15.400000,0.0
3,M,1991-12-19,NaN,2018-01-01,06:53:53,06:59:45,10 - Ministério dos Transportes,6 - Rodoviária,5.866667,0.0
4,M,1969-03-03,DF,2018-01-01,06:58:56,17:40:04,15 - Brasil 21,11 - Rodoviária 2,641.133333,1.0


In [150]:
# Missing values in the whole DataFrame 
df.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

## Attention

Watch out for values that say NaN but are actually a string "NaN", or a string " ", rather than an actual missing value.
Make sure to understand the context. Perhaps the data was not being gathered before, but later on it started being recorded.


In [151]:
df.dtypes
# there seems to be an outlier in ride duration, with a very high max value of 999.6 minutes (more than 15 hours!)
df.describe()

,ride_duration,ride_late
count,214148.000000,214148.000000
mean,29.915932,0.098829
std,58.857297,0.298433
min,3.000000,0.000000
25%,8.083333,0.000000
50%,14.200000,0.000000
75%,33.766667,0.000000
max,999.600000,1.000000


In [152]:
# Missing values in percentage values so we can understand the impact of missing data
df.isnull().sum()/ df.shape[0] # df.shape[0] gives us the first value in the tuple, which is the total number of rows.


user_gender       0.001378
user_birthdate    0.000003
user_residence    0.626144
ride_date         0.000000
time_start        0.000000
time_end          0.150650
station_start     0.000000
station_end       0.000000
ride_duration     0.254676
ride_late         0.254676
dtype: float64

## Significance of missing values

In this dataset user_residence has 62% of missing values. This is a significant proportion that can affect analysis later on. If we remove this many rows we might be removing important data in other columns. In this case a better approach is to understand what happened and see if we can fill in the missing values. 
User gender, on the other hand, has less than 1% missing values, for example, and wouldn't greatly affect analysis. We can get rid of missing values and it wouldn't affect too many rows. 

In [153]:
df_clean = df.dropna(subset=['user_gender'])

# Comparing the number of rows before and after cleaning

print(f"Before cleaning: {df.shape[0]} \nAfter cleaning: {df_clean.shape[0]}")

Before cleaning: 287322 
After cleaning: 286926


In [154]:
# Now the user_gender column has no missing values
df_clean.isnull().sum()


user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration      73064
ride_late          73064
dtype: int64

In [155]:
# Let's fill in missing values in other columns. We can fill in ride duration with the median ride duration, which is not affected by outliers.

df_clean = df_clean.fillna({'ride_duration': df_clean.ride_duration.median()})

# Checking missing values again 
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration          0
ride_late          73064
dtype: int64

In [156]:
# We can fill in values based on the mode, i.e. the most frequent value in the column.
df = df.copy()
df.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [157]:
# M is the mode in the user_gender column 
df.user_gender.value_counts()

user_gender
M    212608
F     74318
Name: count, dtype: int64

In [158]:
# The mode function returns a series. To get the value we need to access the first value in the series, which is [0]. 
df.user_gender.mode(dropna=True)

0    M
Name: user_gender, dtype: object

In [159]:
# Filling the missing values with the mode. Using the mode() we get the first item in the series, which is the value we're looking for.
df = df.fillna({'user_gender': df.user_gender.mode(dropna=True)[0]})
df.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64